In [6]:
class Tree:
    def leaf(data):
        return Tree(data=data)
    def __repr__(self):
        if self.is_leaf():
            return "Leaf(%r)" % self.data
        else:
            return "Tree(%r) { left = %r, right = %r }" % (self.data, self.left, self.right) 
            
    def __init__(self, *, data = None, left = None, right = None):
        self.data = data
        self.left = left
        self.right = right
    
    def is_leaf(self):
        return self.left == None and self.right == None

    def children(self):
        return [x for x in [self.left, self.right] if x]

    def depth(self):
        return max([x.depth() for x in self.children()], default=0) + 1

tree = Tree(data='isSystems?')
tree.left = Tree.leaf(data='like')
tree.right = Tree.leaf(data='TakenOtherSys?')
l1 = Tree.leaf('like')
l2 = Tree.leaf('like')
l3 = Tree.leaf('nah')
l4 = Tree.leaf('nah')
l5 = Tree.leaf('like')
tree4 = Tree(data='likeOtherSys',left=l3,right=l2)
tree3 = Tree(data='morning',left=l5,right=l4)
tree2 = Tree(data='takenOtherSys?', left=tree3, right=tree4)
tree1 = Tree(data='isSystems?', left=l1, right=tree2)


In [7]:
tree1

Tree('isSystems?') { left = Leaf('like'), right = Tree('takenOtherSys?') { left = Tree('morning') { left = Leaf('like'), right = Leaf('nah') }, right = Tree('likeOtherSys') { left = Leaf('nah'), right = Leaf('like') } } }

In [2]:
'''In your python code, load the following dataset and add a boolean "ok" column,
where "True" means the rating is non-negative and "False" means the rating is negative'''
import pandas as pd
dataset = pd.read_csv('Рабочий стол/dataset.csv')
values = []
for i in dataset['rating']:
    if i>0:
        values.append(True)
    else:
        values.append(False)
dataset['ok'] = values

In [3]:
dataset

,rating,easy,ai,systems,theory,morning,ok
0,2,True,True,False,True,False,True
1,2,True,True,False,True,False,True
2,2,False,True,False,False,False,True
3,2,False,False,False,True,False,True
4,2,False,True,True,False,True,True
5,1,True,True,False,False,False,True
6,1,True,True,False,True,False,True
7,1,False,True,False,True,False,True
8,0,False,False,False,False,True,False
9,0,True,False,False,True,True,False


In [32]:
#df[[feature, goal]].value_counts()    
# это мы оцениваем одну фичу или один вопрос
def single_feature_score(data, goal, feature):
    equal = 0
    for i in range(len(feature)):
        if feature[i]==goal[i]:
            equal += 1
            
    return equal

single_feature_score(dataset, dataset['ok'], dataset['easy'])


10

In [31]:
# тут мы перебираем все фичи и выбираем самую лучшую
columns = list(dataset)
score_list = []
feature_dict = {}
for i in columns:
    if i != 'ok' and i != 'rating':
        #best_feature = dataset[i]
        feature_score = single_feature_score(dataset, dataset['ok'], dataset[i])
        feature_dict[i] = feature_score
        score_list.append(feature_score)
        best = max(score_list)
    for k, item in feature_dict.items():
        if item == best:
            best_feature = k
print(best_feature)

ai


In [41]:
def best_feature(data, goal):
    columns = list(dataset)
    score_list = []
    feature_dict = {}
    for i in columns:
        if i != 'ok' and i != 'rating':
            feature_score = single_feature_score(dataset, goal, dataset[i])
            feature_dict[i] = feature_score
            score_list.append(feature_score)
            best = max(score_list)
            worst = min(score_list)
        for k, item in feature_dict.items():
            if item == best:
                best_feature = k
            if item == worst:
                worst_feature = k
    return best_feature, worst_feature
best_feature(dataset, dataset['ok'])[0] # 'ai'
#best_feature(dataset, dataset['ok'])[1] # 'systems'

'ai'

In [55]:
# собираем входные данные для дерева решений
columns = list(dataset)
remaining_features = []
for i in columns:
    if i != 'ok' and i != 'rating':
        remaining_features.append(i)
#print(remaining_features)

#guess←most frequent answer in dataset
true = []
false = []
for i in remaining_features:
    for value in dataset[i]:
        if value == True:
            true.append(value)
        else:
            false.append(value)
f = len(false)
t = len(true)
print(f)
print(t)

50
50


In [43]:
def DecisionTreeTrain(dataset,remaining_features):
    #guess←most frequent answer in dataset
    for i in remaining_features:
    for value in dataset[i]:
        if value == True:
            true.append(value)
        else:
            false.append(value)
    if len(true) > len(false):
        guess = True
    else:
        guess = False
    if 
    if len(remaining_features) = 0:
        return
        
    YES = 0
    NO = 0
    for i in dataset[feature]:
        if i == True:
            YES +=1
        else:
            NO+=1
    


In [ ]:
def DecisionTreeTest(tree, testpoint):
    